In [1]:
import shutil

shutil.make_archive("data", "zip", root_dir="/home/jovyan/lrec_2026/data")



'/home/jovyan/lrec_2026/data.zip'

In [2]:
iate="/home/jovyan/lrec_2026/IATE_export.csv"


In [9]:
import csv

# Archivos de entrada y salida
input_file = iate
output_file_en_csv = "/home/jovyan/lrec_2026/data_10/natural_sciences/extracted_natural_sciences_terms_en.csv"
output_file_es_csv = "/home/jovyan/lrec_2026/data_10/natural_sciences/extracted_natural_sciences_terms_es.csv"

glossary_en_txt = "/home/jovyan/lrec_2026/data_10/natural_sciences/glossary_natural_sciences_en.txt"
glossary_es_txt = "/home/jovyan/lrec_2026/data_10/natural_sciences/glossary_natural_sciences_es.txt"

# Abrimos el CSV original
with open(input_file, mode="r", encoding="utf-8") as infile:
    reader = csv.DictReader(infile, delimiter="|")

    # Filtramos filas cuyo E_DOMAINS sea exactamente "oil industry"
    filtered_rows = [row for row in reader if row["E_DOMAINS"].strip().lower() == "Natural Sciences"]

# Separamos por idioma
rows_en = [row for row in filtered_rows if row["L_CODE"] == "en"]
rows_es = [row for row in filtered_rows if row["L_CODE"] == "es"]

# Guardamos los CSV separados
with open(output_file_en_csv, mode="w", encoding="utf-8", newline="") as outfile_en:
    writer = csv.DictWriter(outfile_en, fieldnames=reader.fieldnames, delimiter="|")
    writer.writeheader()
    writer.writerows(rows_en)

with open(output_file_es_csv, mode="w", encoding="utf-8", newline="") as outfile_es:
    writer = csv.DictWriter(outfile_es, fieldnames=reader.fieldnames, delimiter="|")
    writer.writeheader()
    writer.writerows(rows_es)

# Creamos los glosarios TXT con los términos (T_TERM)
with open(glossary_en_txt, mode="w", encoding="utf-8") as f_en:
    for row in rows_en:
        f_en.write(row["T_TERM"].strip() + "\n")

with open(glossary_es_txt, mode="w", encoding="utf-8") as f_es:
    for row in rows_es:
        f_es.write(row["T_TERM"].strip() + "\n")

print(f"Se han extraído {len(rows_en)} filas en inglés a {output_file_en_csv} y creado glosario en {glossary_en_txt}")
print(f"Se han extraído {len(rows_es)} filas en español a {output_file_es_csv} y creado glosario en {glossary_es_txt}")


Se han extraído 2115 filas en inglés a /home/jovyan/lrec_2026/data_10/education/extracted_education_terms_en.csv y creado glosario en /home/jovyan/lrec_2026/data_10/education/glossary_education_en.txt
Se han extraído 569 filas en español a /home/jovyan/lrec_2026/data_10/education/extracted_education_terms_es.csv y creado glosario en /home/jovyan/lrec_2026/data_10/education/glossary_education_es.txt


In [8]:
import json
import csv

# Ruta al archivo JSON completo con todos los términos
input_path = "/content/drive/My Drive/Seguimiento-Lucía/papers/lrec_2026/data/data_10/info_data/iate_info_data_trade_es.json"

# Ruta de salida en CSV
output_path = "/content/drive/My Drive/Seguimiento-Lucía/papers/lrec_2026/data/data_10/info_data/glossary_info_data_with_definitions_es.csv"

# Cargar el JSON original
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Lista de filas para el CSV
rows = []

for term, content in data.items():
    try:
        items = content.get("items", [])
        for item in items:
            lang_entry = item.get("language", {}).get("es", {})
            definition = lang_entry.get("definition", {})
            if definition and "value" in definition:
                definition_text = definition["value"].strip()

                # Extraer todas las fuentes
                references = definition.get("references", [])
                sources = [ref.get("text", "").strip() for ref in references]
                sources_text = " | ".join(sources) if sources else ""

                rows.append([term, definition_text, sources_text])
    except Exception as e:
        print(f"Error procesando {term}: {e}")

# Guardar en CSV
with open(output_path, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["term", "definition", "source"])
    writer.writerows(rows)

print(f"CSV con definiciones guardado en: {output_path}")


CSV con definiciones guardado en: /content/drive/My Drive/Seguimiento-Lucía/papers/lrec_2026/data/data_10/international_trade/glossary_international_trade_with_definitions_es.csv
